In [1]:
# Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.linear_model import *

In [2]:
# Loading datasets
domain_properties_data = pd.read_csv("data/domain_properties.csv")
sydney_suburbs_data = pd.read_csv("data/syd_sub_rev.csv")

In [3]:
# Understanding basic structure
print("DOMAIN PROPERTIES")
print("shape: ", domain_properties_data.shape, "\n")
print("information: ", domain_properties_data.info(), "\n")
print("first 5 rows: ", domain_properties_data.head(), "\n")
print("column names: ", domain_properties_data.columns, "\n")

print("\n\n\n")

print("SYDNEY SUBURBS REVIEW")
print("shape: ", sydney_suburbs_data.shape, "\n")
print("information: ", sydney_suburbs_data.info(), "\n")
print("first 5 rows: ", sydney_suburbs_data.head(), "\n")
print("column names: ", sydney_suburbs_data.columns, "\n")

DOMAIN PROPERTIES
shape:  (11160, 17) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11160 entries, 0 to 11159
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   price                     11160 non-null  int64  
 1   date_sold                 11160 non-null  object 
 2   suburb                    11160 non-null  object 
 3   num_bath                  11160 non-null  int64  
 4   num_bed                   11160 non-null  int64  
 5   num_parking               11160 non-null  int64  
 6   property_size             11160 non-null  int64  
 7   type                      11160 non-null  object 
 8   suburb_population         11160 non-null  int64  
 9   suburb_median_income      11160 non-null  int64  
 10  suburb_sqkm               11160 non-null  float64
 11  suburb_lat                11160 non-null  float64
 12  suburb_lng                11160 non-null  float64
 13  suburb_elevation     

In [4]:
# Cleaning the datasets
domain_properties_data.columns = domain_properties_data.columns.str.strip().str.lower().str.replace(" ", "_")
sydney_suburbs_data.columns = sydney_suburbs_data.columns.str.strip().str.lower().str.replace(" ", "_")

sydney_suburbs_data.rename(columns={"name":"suburb"}, inplace=True)
# Merge the datasets
Sydney_merged = domain_properties_data.merge(sydney_suburbs_data, on="suburb", how="left")

Sydney_merged["date_sold"] = pd.to_datetime(Sydney_merged["date_sold"], format="%d/%m/%y", errors="coerce")
print("Shape:", Sydney_merged.shape, "\n")
print("Feature data types: ", Sydney_merged.dtypes, "\n")
print("Nulls:\n", Sydney_merged.isnull().sum().sort_values(ascending=False).head(10), "\n")

Shape: (11160, 46) 

Feature data types:  price                                                     int64
date_sold                                        datetime64[ns]
suburb                                                   object
num_bath                                                  int64
num_bed                                                   int64
num_parking                                               int64
property_size                                             int64
type                                                     object
suburb_population                                         int64
suburb_median_income                                      int64
suburb_sqkm                                             float64
suburb_lat                                              float64
suburb_lng                                              float64
suburb_elevation                                          int64
cash_rate                                               float6

In [5]:
Sydney_merged.to_csv("data/Sydney_Housing_Merged.csv", index=False, errors="ignore")

In [6]:
cols_to_drop = [
    "postcode", "ethnic_breakdown_2016", "nearest_train_station", "highlights/attractions", "ideal_for", "traffic", "suburb_lat", "suburb_lng", "region", "review_link"
]

In [7]:
Sydney_merged.drop(columns=cols_to_drop, inplace=True, errors="ignore")
Sydney_merged.columns

Index(['price', 'date_sold', 'suburb', 'num_bath', 'num_bed', 'num_parking',
       'property_size', 'type', 'suburb_population', 'suburb_median_income',
       'suburb_sqkm', 'suburb_elevation', 'cash_rate',
       'property_inflation_index', 'km_from_cbd', 'population_(rounded)*',
       'median_house_price_(2020)', 'median_house_price_(2021)', '%_change',
       'median_house_rent_(per_week)', 'median_apartment_price_(2020)',
       'median_apartment_rent_(per_week)', 'public_housing_%',
       'avg._years_held', 'time_to_cbd_(public_transport)_[town_hall_st]',
       'time_to_cbd_(driving)_[town_hall_st]', 'public_transport',
       'affordability_(rental)', 'affordability_(buying)', 'nature', 'noise',
       'things_to_see/do', 'family-friendliness', 'pet_friendliness', 'safety',
       'overall_rating'],
      dtype='object')

In [8]:
# Checking numeric correlation with target variable price
num_cols = Sydney_merged.select_dtypes(include=["float64", "int64"]).columns
corr = Sydney_merged[num_cols].corr()["price"].sort_values(ascending=False)
print(corr)
print(Sydney_merged.shape)

price                       1.000000
num_bath                    0.395310
suburb_median_income        0.365452
num_bed                     0.314329
property_inflation_index    0.229929
num_parking                 0.228236
safety                      0.139487
nature                      0.136236
noise                       0.129818
family-friendliness         0.126623
pet_friendliness            0.126469
overall_rating              0.113188
things_to_see/do            0.109225
avg._years_held             0.105331
property_size               0.099212
public_transport            0.091391
affordability_(rental)      0.056713
affordability_(buying)      0.003874
suburb_elevation           -0.008412
suburb_population          -0.038954
suburb_sqkm                -0.153286
cash_rate                  -0.236636
km_from_cbd                -0.358321
Name: price, dtype: float64
(11160, 36)


In [9]:
low_corr_cols = [col for col in corr.index if abs(corr[col]) < 0.05]
Sydney_merged.drop(columns=low_corr_cols, inplace=True, errors="ignore")
print(Sydney_merged.shape)

(11160, 33)


In [11]:
# Handling missing values
missing = Sydney_merged.isnull().sum().sort_values(ascending=False)
missing

median_apartment_price_(2020)                    5361
median_apartment_rent_(per_week)                 4641
avg._years_held                                  4305
median_house_rent_(per_week)                     4178
median_house_price_(2021)                        4122
time_to_cbd_(public_transport)_[town_hall_st]    4098
public_housing_%                                 4070
overall_rating                                   4059
safety                                           4059
public_transport                                 4059
time_to_cbd_(driving)_[town_hall_st]             4059
pet_friendliness                                 4059
family-friendliness                              4059
things_to_see/do                                 4059
noise                                            4059
nature                                           4059
affordability_(rental)                           4059
median_house_price_(2020)                        4056
%_change                    

In [14]:
for col in Sydney_merged.columns:
    Sydney_merged[col] = pd.to_numeric(Sydney_merged[col], errors="coerce")
    if 

price                                            False
date_sold                                        False
suburb                                           False
num_bath                                         False
num_bed                                          False
num_parking                                      False
property_size                                    False
type                                             False
suburb_median_income                             False
suburb_sqkm                                      False
cash_rate                                        False
property_inflation_index                         False
km_from_cbd                                      False
population_(rounded)*                             True
median_house_price_(2020)                         True
median_house_price_(2021)                         True
%_change                                          True
median_house_rent_(per_week)                      True
median_apa

In [ ]:
Sydney_merged.drop(columns=[
    "review_link",
    "highlights/attractions",
    "ideal_for",
    "nearest_train_station",
    "things_to_see/do"  # if present
], inplace=True, errors="ignore")

Sydney_merged = Sydney_merged[Sydney_merged["price"].notnull() & (Sydney_merged["price"] > 0)]
Sydney_merged.dropna(subset=["num_bed", "num_bath", "num_parking", "type", "km_from_cbd"], inplace=True)

median_cols = [
    "property_size",
    "suburb_median_income",
    "avg._years_held",
    "median_house_price_(2021)",
    "median_house_rent_(per_week)",
    "median_apartment_price_(2020)",
    "median_apartment_rent_(per_week)"
]

for col in median_cols:
    if col in Sydney_merged.columns:
        Sydney_merged[col] = (
            Sydney_merged[col]
            .astype(str)
            .str.replace("$", "", regex=False)
            .str.replace(",", "", regex=False)
        )
        Sydney_merged[col] = pd.to_numeric(Sydney_merged[col], errors="coerce")

mode_cols = [
    "traffic",
    "safety",
    "overall_rating",
    "family-friendliness",
    "public_housing_%"
]

for col in mode_cols:
    if col in Sydney_merged.columns:
        Sydney_merged[col].fillna(Sydney_merged[col].mode()[0], inplace=True)
        
print(Sydney_merged.shape)
print(Sydney_merged.isnull().sum().sort_values(ascending=False).head(10))

In [ ]:
impute_cols = [
    "avg._years_held",
    "median_house_price_(2021)",
    "median_house_rent_(per_week)",
    "median_apartment_price_(2020)",
    "median_apartment_rent_(per_week)"
]

for col in impute_cols:
    if col in Sydney_merged.columns:
        Sydney_merged[col].fillna(Sydney_merged[col].median(), inplace=True)

mode_cols = [
    "time_to_cbd_(public_transport)_[town_hall_st]",
    "time_to_cbd_(driving)_[town_hall_st]",
    "public_transport",
    "affordability_(rental)",
    "noise"
]

for col in mode_cols:
    if col in Sydney_merged.columns:
        Sydney_merged[col].fillna(Sydney_merged[col].mode()[0], inplace=True)

In [ ]:
print(Sydney_merged.shape)
print(Sydney_merged.isnull().sum().sort_values(ascending=False).head(10))

In [ ]:
drop_cols = [
    "ethnic_breakdown_2016",  # unstructured text
    "region",                 # mostly redundant with suburb
    "population_(rounded)*",  # overlaps with existing population
    "pet_friendliness",       # subjective, sparse
    "nature",                 # subjective, sparse
]
Sydney_merged.drop(columns=drop_cols, inplace=True)

In [ ]:
Sydney_merged.drop(columns=["median_house_price_(2020)", "%_change"], inplace=True)
print(Sydney_merged.shape)
print(Sydney_merged.isnull().sum().sort_values(ascending=False).head(10))

In [ ]:
Sydney_merged.columns

In [ ]:
plt.figure(figsize=(10,10))
sns.histplot(np.log10(Sydney_merged["price"]), bins=50, kde=True)
plt.title("Price Distribution")
plt.xlabel("Price")
plt.ylabel("Count")
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,8))
sns.countplot(x="num_bed", data=Sydney_merged, ax=axes[0])
axes[0].set_title("Number of Beds")

sns.countplot(x="num_bath", data=Sydney_merged, ax=axes[1])
axes[1].set_title("Number of Bathrooms")

sns.countplot(x="num_parking", data=Sydney_merged, ax=axes[2])
axes[2].set_title("Number of Parking")

plt.tight_layout()
plt.show()

In [ ]:
# Log-transform price (visualisation only)
Sydney_merged["log_price"] = np.log10(Sydney_merged["price"])

plt.figure(figsize=(20, 10))
sns.boxplot(x="type", y="log_price", data=Sydney_merged)
plt.xticks(rotation=90)
plt.title("Log(Price) by Property Type")
plt.show()

In [ ]:
# Scatter: Price vs Distance to CBD
plt.figure(figsize=(10, 6))
sns.scatterplot(x="km_from_cbd", y="price", data=Sydney_merged)
plt.title("Price vs Distance to CBD")
plt.xlabel("Kilometres from CBD")
plt.ylabel("Price")
plt.show()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(Sydney_merged.corr(numeric_only=True), annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
Sydney_merged.drop(columns=["log_price"], inplace=True, errors="ignore")

Sydney_encoded = pd.get_dummies(Sydney_merged, columns=["type"], drop_first=True)
Sydney_encoded.columns

In [ ]:
X = Sydney_encoded.drop(columns=["price"])
y = Sydney_encoded["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
Sydney_encoded.dtypes

In [26]:
X_train = X_train.drop(columns=["suburb", "date_sold"], errors="ignore")
X_test = X_test.drop(columns=["suburb", "date_sold"], errors="ignore")

cols_to_float = [
    "public_housing_%",
    "time_to_cbd_(public_transport)_[town_hall_st]", "time_to_cbd_(driving)_[town_hall_st]"
    ]

for col in cols_to_float:
    X_train[col] = pd.to_numeric(X_train[col], errors="coerce")
    X_test[col] = pd.to_numeric(X_test[col], errors="coerce")

bool_cols = Sydney_encoded.select_dtypes("bool").columns
X_train[bool_cols] = X_train[bool_cols].astype(int)
X_test[bool_cols] = X_test[bool_cols].astype(int)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)